In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import files
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
!pip install gdown
!pip install --upgrade gdown
!pip install patool
import patoolib

os.mkdir('unzipped_folder')

!gdown "https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download"

patoolib.extract_archive("/content/dataset.zip", outdir="/content/unzipped_folder")

ratings = pd.read_csv("/content/unzipped_folder/dataset/rating.csv")
animes = pd.read_csv("/content/unzipped_folder/dataset/anime.csv")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 4.2 MB/s eta 0:00:00
Downloading...
From (uriginal): https://drive.google.com/u/0/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download
From (redirected): https://drive.google.com/uc?id=1mipJf49ZvqD3CArtutXNOJriqdtb9tAH&export=download&confirm=t&uuid=3118bbee-e0fe-4e5a-8194-1075435da94e
To: /content/dataset.zip
100% 26.3M/26.3M [00:00<00:00, 171MB/s]
patool: Extracting /content/dataset.zip ...
patool: running /usr/bin/7z x -o/content/unzipped_folder -- /content/dataset.zip
patool: ... /con

In [3]:
ratings_2 = ratings[ratings.rating != -1]

def get_animes(user_id):

  select = ratings.loc[ratings['user_id'] == user_id]
  all_animes = select['anime_id'].tolist()

  return all_animes


def get_top_rated_animes(user_id):

  select = ratings_2.loc[ratings_2['user_id'] == user_id]
  sel = select[['anime_id', 'rating']]
  sel_sorted = sel.sort_values(by = 'rating', ascending = False)
  rated_animes = sel_sorted['anime_id'].tolist()
  top_rated_animes = rated_animes[0:5]

  return top_rated_animes


def create_matrix(df):
	
	N = len(df['user_id'].unique())
	M = len(df['anime_id'].unique())
	
	user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
	anime_mapper = dict(zip(np.unique(df["anime_id"]), list(range(M))))
	
	user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
	anime_inv_mapper = dict(zip(list(range(M)), np.unique(df["anime_id"])))
	
	user_index = [user_mapper[i] for i in df['user_id']]
	anime_index = [anime_mapper[i] for i in df['anime_id']]

	X = csr_matrix((df["rating"], (anime_index, user_index)), shape=(M, N))
	
	return X, user_mapper, anime_mapper, user_inv_mapper, anime_inv_mapper


X, user_mapper, anime_mapper, user_inv_mapper, anime_inv_mapper = create_matrix(ratings_2)

kNN = NearestNeighbors(n_neighbors=10, algorithm="brute", metric='cosine')
kNN.fit(X)

def find_similar_animes(anime_id, k, show_distance=False):
      
    neighbour_ids = []

    anime_ind = anime_mapper[anime_id]
    anime_vec = X[anime_ind]
    anime_vec = anime_vec.reshape(1,-1)

    neighbour = kNN.kneighbors(anime_vec, return_distance=show_distance)

    for i in range(0,k):

        n = neighbour.item(i)
        neighbour_ids.append(anime_inv_mapper[n])

    neighbour_ids.pop(0)

    return neighbour_ids


act_users = ratings_2['user_id'].unique().tolist()

ids_names = dict(zip(animes['anime_id'], animes['name']))

In [4]:
user_id = int(input("Enter user id: "))

if user_id in act_users:
  
  all_animes = get_animes(user_id)
  top_rated_animes = get_top_rated_animes(user_id)

  recommended_animes = []

  for i in top_rated_animes:

      similar_animes = find_similar_animes(i, k=10)
      
      for j in similar_animes:
        
          if (j not in recommended_animes) and (j not in all_animes):
            recommended_animes.append(j)

            
  recommended_animes = recommended_animes[0:10]
  
  print("")
  print("Recommended animes:")
  print("")

  for i in recommended_animes:
    
    print(ids_names[i])


else :

  print("")
  print("Cold start!")
  print("This user has not rated any animes, so the system cannot recommend an anime.")

Enter user id: 3

Recommended animes:

Pokemon: Mewtwo no Gyakushuu
Pokemon: Maboroshi no Pokemon Lugia Bakutan
Pokemon: Kesshoutou no Teiou Entei
Kill la Kill
Dragon Ball Z Movie 14: Kami to Kami
Code Geass: Hangyaku no Lelouch
Code Geass: Hangyaku no Lelouch R2
Elfen Lied
Fullmetal Alchemist
Angel Beats!
